In [ ]:
!pip install gradio

In [ ]:
from google.colab import drive
import sys
import pandas as pd

import gradio as gr

In [ ]:
drive.mount('/content/drive')
sys.path.append('/content/drive/Shareddrives/CMPE260/')

Mounted at /content/drive


In [ ]:
%run /content/drive/Shareddrives/CMPE260/code/rec_implementation.ipynb
%run /content/drive/Shareddrives/CMPE260/code/rec_implementation_w_features.ipynb

cpu
cpu


### Demo

In [ ]:
def generate_user_books(dummy_argument=None):
  random_user = test_data['user'].sample().iloc[0]
  temp_test_data = test_data[test_data.user == random_user]
  temp_filtered = pd.merge(temp_test_data, filtered_df, on=['user', 'book'])
  temp_filtered

  features = user_features_df[user_features_df['user'] == random_user].iloc[0, 2:4]

  temp_filtered['age'] = features[0]
  temp_filtered['country'] = features[1]

  books = temp_filtered['book'].sample(2)

  book1 = temp_filtered[temp_filtered.book == books.iloc[0]]
  book2 = temp_filtered[temp_filtered.book == books.iloc[1]]

  user = f'User: {random_user}, Age: {book1.iloc[0, 8]}, Country: {book1.iloc[0, 9]}\n'
  book1_string = book1[['book', 'title', 'author']].to_string()+f'\n'
  book2_string = book2[['book', 'title', 'author']].to_string()+f'\n'

  # book1_string = f'Book 1: Id- {book1.iloc[0, 1]}, Title- {book1.iloc[0, 3]}, Author- {book1.iloc[0, 4]}\n'
  # book2_string = f'Book 2: Id- {book2.iloc[0, 1]}, Title- {book2.iloc[0, 3]}, Author- {book2.iloc[0, 4]}\n',

  input_2 = f'{book1.iloc[0, 1]}, {book2.iloc[0, 1]}'
  results = generate_recommendations(input_2,random_user)

  return user+book1_string+book2_string, results

In [ ]:
def run_model_with_features(model_dir, book1, book2, random_user):
  recommendations = []

  state_rep = State_Representation_w_Features(num_of_countries, num_of_age_buckets, num_of_books, params['embedding_dim'], params['hidden_dim'])
  policy = Actor(params['embedding_dim'], params['hidden_dim'])
  state_rep.load_state_dict(torch.load(params[model_dir] + 'best_state_repr.pth'))
  policy.load_state_dict(torch.load(params[model_dir] + 'best_policy_net.pth'))

  for model, state_representation in zip([policy], [state_rep]):
    env = Environment_w_Features(test_matrix)
    user, country, age, memory = env.reset(random_user)

    # give 2 books that the user has rated in the past
    user, memory, reward, _ = env.step(torch.tensor([book1]))
    user, memory, reward, _ = env.step(torch.tensor([book2]))

    # get next 3 recommendations based on the two books above
    recs = []
    for i in range(3):
        action_emb = model(state_representation(country, age, memory))
        action = model.get_action(
            user,
            torch.tensor(env.memory[user.detach().cpu().numpy().astype(int), :]),
            state_representation,
            action_emb,
            torch.tensor([item for item in env.available_books if item not in env.viewed_books]).long()
        )
        user, memory, reward, _ = env.step(action)
        recs.append(action)

    recommendations.append(recs)

    recos = [r.item() for r in recommendations[0]]
    return recos

In [ ]:
def run_model_vanilla(model_dir, book1, book2, random_user):
  recommendations = []

  state_rep = State_Representation(num_of_users, num_of_books, params['embedding_dim'], params['hidden_dim'])
  policy = Actor(params['embedding_dim'], params['hidden_dim'])
  state_rep.load_state_dict(torch.load(params[model_dir] + 'best_state_repr.pth'))
  policy.load_state_dict(torch.load(params[model_dir] + 'best_policy_net.pth'))

  for model, state_representation in zip([policy], [state_rep]):
    env = Environment(test_matrix)
    user, memory = env.reset(random_user)

    # give 2 books that the user has rated in the past
    user, memory, reward, _ = env.step(torch.tensor([book1]))
    user, memory, reward, _ = env.step(torch.tensor([book2]))

    # get next 3 recommendations based on the two books above
    recs = []
    for i in range(3):
        action_emb = model(state_representation(user, memory))
        action = model.get_action(
            user,
            torch.tensor(env.memory[user.detach().cpu().numpy().astype(int), :]),
            state_representation,
            action_emb,
            torch.tensor([item for item in env.available_books if item not in env.viewed_books]).long()
        )
        user, memory, reward, _ = env.step(action)
        recs.append(action)

    recommendations.append(recs)

    recos = [r.item() for r in recommendations[0]]
    return recos

In [ ]:
def generate_recommendations(books, random_user):
  books = books.split(',')
  book1 = int(books[0].strip())
  book2 = int(books[1].strip())

  model_dirs = ['log_w_features_dir']
  methods = ['Vanilla with features']

  vanilla_model_dirs = ['log_base_dir', 'log_w_ou_dir', 'log_w_gaussian_dir']
  vanilla_methods = ['Vanilla', 'Vanilla with OU Noise', 'Vanilla with Gaussian Noise']

  rec_string = ""

  for d, m in zip(model_dirs, methods):
    recs = run_model_with_features(d, book1, book2, random_user)
    recommended_books = filtered_df[filtered_df.book.isin(recs)][['title', 'author', 'best_seller']].drop_duplicates()

    rec_string = rec_string + m + "\n"  + recommended_books.to_string() + "\n\n"

  for d, m in zip(vanilla_model_dirs, vanilla_methods):
    recs = run_model_vanilla(d, book1, book2, random_user)
    recommended_books = filtered_df[filtered_df.book.isin(recs)][['title', 'author', 'best_seller']].drop_duplicates()

    rec_string = rec_string + m + "\n"  + recommended_books.to_string() + "\n\n"

  return rec_string

In [ ]:
generate_user = gr.Interface(
    fn=generate_user_books,
    inputs=gr.components.Button(value="Generate book history for a random user"),
    outputs=[
        gr.components.Textbox(label = "Book history of random user",lines=10, placeholder="User book history will appear here..."),
        gr.components.Textbox(label="Recommended books",lines=10, placeholder="Book recommendations")
    ],
    title="Generate user book history",
)

generate_user.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d6106d509fca74dc20.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d6106d509fca74dc20.gradio.live
